<a href="https://colab.research.google.com/github/DrAnonim/CoinMarketCap-API/blob/master/Copy_of_clickHouse_test_colab_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

In [ ]:
!pip install clickhouse-driver

     |████████████████████████████████| 481kB 8.4MB/s 


In [ ]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('default',), ('system',)]

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
df = pd.DataFrame({'dt':['2020-01-01','2020-01-02'],'product':['product1','product2'],'amount':[2,3]})

In [ ]:
df

,dt,product,amount
0,2020-01-01,product1,2
1,2020-01-02,product2,3


In [ ]:
df['dt'] = pd.to_datetime(df['dt'],format='%Y-%m-%d')

In [ ]:
client.execute('DROP DATABASE IF EXISTS db')

[]

In [ ]:
client.execute('CREATE DATABASE db')

[]

In [ ]:
client.execute('SHOW DATABASES')

[('db',), ('default',), ('system',)]

In [ ]:
# Старый вариант settings={'use_numpy': True}
client = Client(host='localhost', user='default', port='9000', database='db',settings={'columnar': True})

In [ ]:
client.execute('DROP TABLE IF EXISTS test')

[]

In [ ]:
client.execute('CREATE TABLE test (dt Date, product String, amount Int32) ENGINE = Memory')

[]

In [ ]:
client.execute('SHOW TABLES FROM db')

[('test',)]

In [ ]:
client.execute("INSERT INTO test VALUES", df.to_dict('records'))

2

In [ ]:
def select(sql):
  return client.query_dataframe(sql)

In [ ]:
sql = '''SELECT *
         FROM test as t
         ORDER BY t.dt
         LIMIT 25'''

In [ ]:
select(sql)

,dt,product,amount
0,2020-01-01,product1,2
1,2020-01-02,product2,3


https://clickhouse.tech/docs/en/getting-started/example-datasets/metrica/

In [ ]:
sql  = '''SELECT COUNT(*) FROM datasets.hits_v1'''

```
Задание 1
В таблице table есть 2 события статистики:
tutorial_finished - окончание туториала
kiss_receive - получение поцелуя - пишется на каждый поцелуй
Нужно вывести среднее время в минутах, которое занимает у
прошедшего туториал юзера, до получения 3 поцелуя. Максимальное
время между событиями нужно ограничить сверху 30 минутами.
У каждого ивента есть поля:
__uid - id юзера
__event_time - время события, формат ‘YY-MM-DD HH-MM-SS’
__event_date - дата события, формат ‘YY-MM-DD’
Работать с полем ивента: - id того, у кого было это событие.
Пример запроса, который выведет время и id юзера, которые
закончили туториал:
select event_time, tutorial_finished__uid
from table
where event = ‘tutorial_finished’
Задание 2
Есть событие статистики income_generated (совершение покупки) c
полями
__uid - id юзера
__event_time - время события, формат ‘YY-MM-DD HH-MM-SS’
__event_date - дата события, формат ‘YY-MM-DD’
__value – сумма покупки
Используя диалект SQL базы данных Clickhouse, необходимо
предложить sql запрос, который бы вывел кумулятивную сумму
покупок с интервалом в 6 часов с начала текущего месяца по сейчас.
```

In [ ]:
user1 = pd.DataFrame({'event_type':['tutorial_finished','kiss_receive','kiss_receive','kiss_receive','kiss_receive'],
                   'uid':[1,1,1,1,1],
                   'event_time':['2021-04-01 10:32:21','2021-04-01 10:41:00',
                                 '2021-04-01 11:10:00','2021-04-01 11:20:00',
                                 '2021-04-01 15:41:00']})

In [ ]:
user2 = pd.DataFrame({'event_type':['tutorial_finished','kiss_receive','kiss_receive','kiss_receive','kiss_receive'],
                   'uid':[2,2,2,2,2],
                   'event_time':['2021-04-01 10:32:21','2021-04-01 10:36:00',
                                 '2021-04-01 11:01:00','2021-04-01 11:30:00',
                                 '2021-04-01 15:40:00']})

In [ ]:
user3 = pd.DataFrame({'event_type':['tutorial_finished','kiss_receive','kiss_receive','kiss_receive','kiss_receive'],
                   'uid':[3,3,3,3,3],
                   'event_time':['2021-04-01 10:32:21','2021-04-01 10:36:00',
                                 '2021-04-01 11:01:00','2021-04-01 11:50:00',
                                 '2021-04-01 12:10:00']})

In [ ]:
user4 = pd.DataFrame({'event_type':['kiss_receive','kiss_receive','kiss_receive','kiss_receive','kiss_receive'],
                   'uid':[4]*5,
                   'event_time':['2021-04-01 10:32:21','2021-04-01 10:36:00',
                                 '2021-04-01 11:01:00','2021-04-01 11:50:00',
                                 '2021-04-01 12:10:00']})

In [ ]:
df = pd.concat([user1,user2,user3,user4])

In [ ]:
df

,event_type,uid,event_time
0,tutorial_finished,1,2021-04-01 10:32:21
1,kiss_receive,1,2021-04-01 10:41:00
2,kiss_receive,1,2021-04-01 11:10:00
3,kiss_receive,1,2021-04-01 11:20:00
4,kiss_receive,1,2021-04-01 15:41:00
0,tutorial_finished,2,2021-04-01 10:32:21
1,kiss_receive,2,2021-04-01 10:36:00
2,kiss_receive,2,2021-04-01 11:01:00
3,kiss_receive,2,2021-04-01 11:30:00
4,kiss_receive,2,2021-04-01 15:40:00


In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
df

,event_type,uid,event_time
0,tutorial_finished,1,2021-04-01 10:32:21
1,kiss_receive,1,2021-04-01 10:41:00
2,kiss_receive,1,2021-04-01 11:10:00
3,kiss_receive,1,2021-04-01 11:20:00
4,kiss_receive,1,2021-04-01 15:41:00
0,tutorial_finished,2,2021-04-01 10:32:21
1,kiss_receive,2,2021-04-01 10:36:00
2,kiss_receive,2,2021-04-01 11:01:00
3,kiss_receive,2,2021-04-01 11:30:00
4,kiss_receive,2,2021-04-01 15:40:00


In [ ]:
df.dtypes

event_type            object
uid                    int64
event_time    datetime64[ns]
dtype: object

In [ ]:
client.execute('DROP TABLE IF EXISTS events')
client.execute('''CREATE TABLE events
(event_type String, uid Int32, event_time DateTime)
ENGINE = Memory''')

[]

In [ ]:
df

,event_type,uid,event_time
0,tutorial_finished,1,2021-04-01 10:32:21
1,kiss_receive,1,2021-04-01 10:41:00
2,kiss_receive,1,2021-04-01 11:10:00
3,kiss_receive,1,2021-04-01 11:20:00
4,kiss_receive,1,2021-04-01 15:41:00
0,tutorial_finished,2,2021-04-01 10:32:21
1,kiss_receive,2,2021-04-01 10:36:00
2,kiss_receive,2,2021-04-01 11:01:00
3,kiss_receive,2,2021-04-01 11:30:00
4,kiss_receive,2,2021-04-01 15:40:00


In [ ]:
client.execute("INSERT INTO events VALUES", df.to_dict('records'))

20

In [ ]:
sql = '''select t.event_type, t.uid, t.event_time, e.event_time from events t
left join events e on t.uid = e.uid
where
--e.event_time < t.event_time and
t.uid = 1

order by t.uid, t.event_time

'''

In [ ]:
select(sql)

,event_type,uid,event_time,e_event_time
0,tutorial_finished,1,2021-04-01 10:32:21,2021-04-01 10:41:00
1,tutorial_finished,1,2021-04-01 10:32:21,2021-04-01 10:32:21
2,tutorial_finished,1,2021-04-01 10:32:21,2021-04-01 11:10:00
3,tutorial_finished,1,2021-04-01 10:32:21,2021-04-01 11:20:00
4,tutorial_finished,1,2021-04-01 10:32:21,2021-04-01 15:41:00
5,kiss_receive,1,2021-04-01 10:41:00,2021-04-01 11:10:00
6,kiss_receive,1,2021-04-01 10:41:00,2021-04-01 10:41:00
7,kiss_receive,1,2021-04-01 10:41:00,2021-04-01 10:32:21
8,kiss_receive,1,2021-04-01 10:41:00,2021-04-01 11:20:00
9,kiss_receive,1,2021-04-01 10:41:00,2021-04-01 15:41:00


In [ ]:
sql = '''select t.event_type, t.uid, t.event_time, max(e.event_time) as event_time_prev from events t
left join events e on t.uid = e.uid
-- where e.event_time is null or e.event_time < t.event_time
group by t.event_type, t.uid, t.event_time
order by t.uid, t.event_time
'''

In [ ]:
select(sql)

,event_type,uid,event_time,event_time_prev
0,tutorial_finished,1,2021-04-01 10:32:21,2021-04-01 15:41:00
1,kiss_receive,1,2021-04-01 10:41:00,2021-04-01 15:41:00
2,kiss_receive,1,2021-04-01 11:10:00,2021-04-01 15:41:00
3,kiss_receive,1,2021-04-01 11:20:00,2021-04-01 15:41:00
4,kiss_receive,1,2021-04-01 15:41:00,2021-04-01 15:41:00
5,tutorial_finished,2,2021-04-01 10:32:21,2021-04-01 15:40:00
6,kiss_receive,2,2021-04-01 10:36:00,2021-04-01 15:40:00
7,kiss_receive,2,2021-04-01 11:01:00,2021-04-01 15:40:00
8,kiss_receive,2,2021-04-01 11:30:00,2021-04-01 15:40:00
9,kiss_receive,2,2021-04-01 15:40:00,2021-04-01 15:40:00


In [ ]:
len(select(sql))

16

In [ ]:
sql = '''
with events_prep as (

select distinct 'dummy' as event_type , t.uid,
toDateTime('1900-01-01 00:00:00') as event_time

from events t

union all

select * from events t),

events_lag as (

select t.event_type, t.uid, t.event_time, max(e.event_time) as event_time_prev from events_prep t
left join events_prep e on t.uid = e.uid
where e.event_time < t.event_time
group by t.event_type, t.uid, t.event_time
order by t.uid, t.event_time),

new_ses as (


select t.*,


dateDiff('minute',t.event_time_prev,t.event_time) as min_since_prev,
case when dateDiff('minute',t.event_time_prev,t.event_time) >= 30 then 1 else 0 end new_session



 from events_lag t),

 ses_id as (

 select
 t.event_type, t.uid, t.event_time,

 sum(ns.new_session) as session_id

 from new_ses t
 left join new_ses ns on t.uid = ns.uid
 where ns.event_time <= t.event_time
 group by t.event_type, t.uid, t.event_time
order by t.uid, t.event_time),


event_group as (


select t.uid, t.session_id,
arrayStringConcat(groupArray(case when t.event_type = 'tutorial_finished' then '0' else '1' end)) as event_group
from ses_id t
group by t.uid,t.session_id
order by t.uid, t.session_id)


select * from event_group t
where t.event_group like '%0111%'

'''

In [ ]:
select(sql)

,uid,session_id,event_group
0,1,1,0111
1,2,1,0111


In [ ]:
sql = '''select * from (select '0100100' as col) t
where t.col like '%0111%'
'''

In [ ]:
select(sql)

""


In [ ]:
select(sql)

,uid,session_id,event_group
0,1,1,0111
1,2,1,0111


In [ ]:
sql = '''
with events_prep as (

select distinct 'dummy' as event_type , t.uid,
toDateTime('1900-01-01 00:00:00') as event_time

from events t

union all

select * from events t),

events_lag as (

select t.event_type, t.uid, t.event_time, max(e.event_time) as event_time_prev from events_prep t
left join events_prep e on t.uid = e.uid
where e.event_time < t.event_time
group by t.event_type, t.uid, t.event_time
order by t.uid, t.event_time),

new_ses as (


select t.*,


dateDiff('minute',t.event_time_prev,t.event_time) as min_since_prev,
case when dateDiff('minute',t.event_time_prev,t.event_time) >= 30 then 1 else 0 end new_session



 from events_lag t),

 ses_id as (

 select
 t.event_type, t.uid, t.event_time,

 sum(ns.new_session) as session_id

 from new_ses t
 left join new_ses ns on t.uid = ns.uid
 where ns.event_time <= t.event_time
 group by t.event_type, t.uid, t.event_time
order by t.uid, t.event_time),


event_group as (


select t.uid, t.session_id,
arrayStringConcat(groupArray(case when t.event_type = 'tutorial_finished' then '0' else '1' end)) as event_group
from ses_id t
group by t.uid,t.session_id
order by t.uid, t.session_id)


select t.uid,t.session_id, min(t.event_time) from ses_id t

'''